In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import re
import csv
import os

In [219]:
website='https://aws.amazon.com/ec2/autoscaling/faqs/'
service=Service(r'C:\Users\aryam\Downloads\chromedriver-win64\chromedriver.exe')

driver=webdriver.Chrome(service=service)
driver.get(website)


wait=WebDriverWait(driver,10)

dialogue_button=wait.until(EC.presence_of_element_located((By.XPATH,"//button[@aria-label='Close']")))

time.sleep(1)

dialogue_button.click()

soup = BeautifulSoup(driver.page_source, "html.parser")
page_topics=[]
container = soup.find("div", class_="eb-sidebar-content")
topics=container.find_all("span")

for topic in topics:
    name_with_counter = topic.find("a").text.strip()
    name=re.sub(r'\d+', '', name_with_counter )
    counter = topic.find("div", class_="eb-sidebar-link-counter").text.strip()
    page_topics.append(f"{name}: {counter}")
    
page_topics = {item.split(': ')[0]: int(item.split(': ')[1]) for item in page_topics}

In [220]:
openall_buttons=driver.find_elements(By.XPATH,"//*[contains(@class, 'section-expander-header-toggle-button')]")

for button in openall_buttons:
    driver.execute_script("arguments[0].click();",button)
    time.sleep(2)

In [221]:
faqs = driver.find_elements(By.XPATH, "//*[contains(@id, 'product-faqs#ec2-auto-scaling-faq#')]")

data=[]
for i in faqs:
    data.append(i.text)

In [223]:
file_name=r'C:\Users\aryam\Desktop\project\faqs.txt'
with open(file_name,"w",newline='') as file:
        for i in range(0,9,2):
            file.write(f"###{data[i]}###\n\n\n")        

In [224]:
input_file = r'C:\Users\aryam\Desktop\project\faqs.txt'
output_folder = r'C:\Users\aryam\Desktop\project\\'
with open(input_file, 'r') as file:
    content = file.read()
blocks = [block.strip() for block in content.split('###') if block.strip()]


In [225]:
for index, (topic, block) in enumerate(zip(page_topics.keys(), blocks), start=1):
    output_file = f"{output_folder}{topic}.txt"
    
    with open(output_file, 'w') as file:
        file.write(block)

In [226]:
driver.quit()